# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\nobel\OneDrive\桌面\BEE5750\hw\hw5-rishi`
Precompiling project...
  ✓ Latexify → DataFramesExt
  1 dependency successfully precompiled in 17 seconds. 232 already precompiled.


In [2]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [11]:
waste_model = Model(HiGHS.Optimizer)
#Decision Variables

I = 1:3
#1=city 1
#2=city 2
#3=city 3
J = 1:3
#1=LF
#2=MRF
#3=WTE
K = 1:3
#1=LF
#2= MRF
#3 = WTE

@variable(waste_model, w[i in I, j in J] >=0)
@variable(waste_model, r[k in K, j in J] >=0)
@variable(waste_model, Y[j])

#Transporation Costs
sum(a*l[i,j]*w[i,j] for i in I, j in J )

#Disposal Costs
sum(Fixed_Costs[j] + Var_Costs[j]*(sum(w[i,j] for i in I) + sum(r[k,j] for k in K)) +  for j in J)

#Costs

#WTE Costs

WTE_Costs = 2500*Y[3] + 60*(w[1,3]+w[2,3]+w[3,3]+r[2,3])

#MRF Costs

MRF_Costs = 1500*Y[2] + 7*(w[1,2]+w[2,2]+w[3,2]) + 40(.4(w[1,2]+w[2,2]+w[3,2]))

#LF Costs

LF_Costs = 2000*Y[1] + 50*(w[1,1]+w[2,1]+w[3,1] + r[2,1] + r[3,1]) 



Trans_Costs = 5(5*w[1,1]+30*w[1,2]+15*w[1,3]+15*w[2,1]+25*w[2,2]+10*w[2,3]+13*w[3,1]+45*w[3,2]+20*w[3,3])

Total_Costs = WTE_Costs + MRF_Costs + LF_Costs


#Constraints

#Disposal Constraints
@constraint(waste_model, WTE_Limit, w[1,3]+ w[2,3] + w[3,3] + r[2,3] <= 210)
@constraint(waste_model, MRF_Limit, w[1,2] + w[2,2] + w[3,2] <= 350)
@constraint(waste_model, LF_Limit, w[1,1]+ w[2,1] + w[3,1] + r[2,1] + r[3,1] <= 200)

#City Mass Balance Constraints
@constraint(waste_model, city1, w[1,1]+ w[1,2] + w[1,3]= 100)
@constraint(waste_model, city2, w[2,1] + w[2,2] + w[2,3] = 90)
@constraint(waste_model, city3, w[3,1]+ w[3,2] + w[3,3] = 120)

#Residual Ash Mass-Balance Constraints
@constraint(waste_model, recycled_ash_flow, r[2,3] = .14(.6*(w[1,2]+w[2,2] + w[3,2])))
@constraint(waste_model, total_ash_flow, r[3,1] = .16(w[1,3] + w[2,3] + w[3,3]) + r[2,3])
@constraint(waste_model, nonrecycled_flow, r[2,1] +r[2,3] = .6(w[1,2] + w[2,2] + w[3,2]))

#Indicator Contraints
@constraint(waste_model,indicator1, Y[1] = 1)
@constraint(waste_model,indicator2, !Y[2] => {w[1,2]+ w[2,2] + w[3,2] = 0})
@constraint(waste_model,indicator2, !Y[3] => {w[1,3]+ w[2,3] + w[3,3] + r[2,3] = 0})







ErrorException: At c:\Users\nobel\OneDrive\桌面\BEE5750\hw\hw5-rishi\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:19: `@variable(waste_model, Y[j])`: unexpected error parsing reference set: j

In [ ]:
waste_model = Model(HiGHS.Optimizer)

# Non-negativity constraints(variable)
@variable(waste_model, W[1:3, 1:3] >= 0)
@variable(waste_model, R[1:3, 1:3] >= 0)
@variable(waste_model, Y[1:3], Bin)

#Objective -> minimize cost
@objective(waste_model, Min, 
    82.5 * W[1,1] + 68 * W[1,2] + 57.5 * W[1,3] + #MRF Costs + LF Costs + WTE Costs = Total Cost
    75 * W[2,1] + 60.5 * W[2,2] + 72.5 * W[2,3] +
    90 * W[3,1] + 90.5 * W[3,2] + 69.5 * W[3,3] +
    77 * R[1,3] + 82.5 * R[2,1] + 98 * R[2,3] + 
    2500 * Y[1] + 1500 * Y[2] + 2000 * Y[3])

# Disposal Limit Constraints
@constraint(waste_model, W[1,1] + W[2,1] + W[3,1] + R[2,1] <= 210)
@constraint(waste_model, W[1,2] + W[2,2] + W[3,2] <= 350)
@constraint(waste_model, W[1,3] + W[2,3] + W[3,3] + R[1,3] + R[2,3] <= 200)

# City Mass Balance Constraints
@constraint(waste_model, W[1,1] + W[1,2] + W[1,3] == 100)
@constraint(waste_model, W[2,1] + W[2,2] + W[2,3] == 90)
@constraint(waste_model, W[3,1] + W[3,2] + W[3,3] == 120)

# Residual Waste Constraints
@constraint(waste_model, R[1,3] == 0.16 * (0.6 * (W[1,2] + W[2,2] + W[3,2])))
@constraint(waste_model, R[2,1] == 0.16 * (0.6 * (W[1,2] + W[2,2] + W[3,2])))
@constraint(waste_model, R[2,3] == 0.14 * (0.4 * (W[1,2] + W[2,2] + W[3,2])))

M = 10000
# Commitment Constraints
@constraint(waste_model, W[1,1] + W[2,1] + W[3,1] + R[2,1] <= M * Y[1])
@constraint(waste_model, W[1,1] + W[2,1] + W[3,1] + R[2,1] >= 0)
@constraint(waste_model, W[2,1] + W[2,2] + W[2,3] <= M * Y[2])
@constraint(waste_model, W[2,1] + W[2,2] + W[2,3] >= 0)
@constraint(waste_model, Y[3] == 1)


optimize!(waste_model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [6e-02, 1e+04]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 4e+02]
Presolving model
10 rows, 14 cols, 39 nonzeros  0s
7 rows, 11 cols, 32 nonzeros  0s
6 rows, 9 cols, 27 nonzeros  0s
5 rows, 9 cols, 21 nonzeros  0s

Solving MIP model with:
   5 rows
   9 cols (0 binary, 0 integer, 0 implied int., 9 continuous)
   21 nonzeros
MIP-Timing:      0.0087 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol     

In [ ]:
@Show value.()

In [14]:
println("Optimal objective value: ", objective_value(model))

println("Y values:")

println("Y1 = ", value(Y[1]))
println("Y2 = ", value(Y[2]))
println("Y3 = ", value(Y[3]))
println("W values:")
for i in 1:3
    for j in 1:3
        println("W[", i, ",", j, "] = ", value(W[i, j]))
    end
end

println("R values:")
for i in 1:3
    for j in 1:3
        println("R[", i, ",", j, "] = ", value(R[i, j]))
    end
end

Optimal objective value: 27250.0
Y values:
Y1 = 1.0
Y2 = 1.0
Y3 = 1.0
W values:
W[1,1] = 0.0
W[1,2] = 0.0
W[1,3] = 100.0
W[2,1] = 90.0
W[2,2] = -0.0
W[2,3] = 0.0
W[3,1] = 20.0
W[3,2] = 0.0
W[3,3] = 100.0
R values:
R[1,1] = 0.0
R[1,2] = 0.0
R[1,3] = 0.0
R[2,1] = 0.0
R[2,2] = 0.0
R[2,3] = 0.0
R[3,1] = 0.0
R[3,2] = 0.0
R[3,3] = 0.0


## References

List any external references consulted, including classmates.